## Library Import

In [2]:
# Library Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\codej\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Utils Function For Preprocessing

In [3]:
# Utils Function

def convert_to_number(series):
    for i  in range(len(series)):
        try:
            series[i] = int(series[i])
        except ValueError:
            series[i] = None
    return series


def remove_html_tags_special_character(col: pd.Series) -> pd.Series:
    tags_list = ['<p>' ,'</p>' , '<p*>',
                 '<ul>','</ul>',
                 '<li>','</li>',
                 '<br>',
                 '<strong>','</strong>',
                 '<span*>','</span>',
                 '<a href*>','</a>',
                 '<em>','</em>','<br>','<br />','<div>','</div>','\\n','~']
    for tag in tags_list:
        col.replace(to_replace=tag,value='',regex=False,inplace=True)
    return col

punctuations_list = string.punctuation
def remove_punctuations(text):
    temp = str.maketrans('', '', punctuations_list)
    text = str(text)
    return text.translate(temp)


def remove_stopwords(text):
    stop_words = stopwords.words('english')
 
    imp_words = []
 
    # Storing the important words
    for word in str(text).split():
        word = word.lower()
 
        if (word not in stop_words) and 'br' not in word:
            imp_words.append(word)
 
    output = " ".join(imp_words)
 
    return output

def balance_data(df,y_column_name):
    ham_msg = df[df[y_column_name] == 0]
    spam_msg = df[df[y_column_name] == 1]
    print(ham_msg.shape)
    print(spam_msg.shape)

    if len(ham_msg) >= len(spam_msg):
        ham_msg.sample(n=len(spam_msg),random_state=42)
    else:
        spam_msg.sample(n=len(ham_msg),random_state=42)
    return pd.concat([ham_msg, spam_msg],ignore_index=True)

## Preprcessor Handler

In [4]:
def preprocess(df,X_column_name: str,y_column_name: str,split_ratio = [0.1,0.2]):
    """
    Preprocess the data for model training
    Arguments:
    df: Dataframe of raw data
    X_column_name: represent the input x column name in DataFrame
    Y_Column_nameL represent the target y column name in DataFrame
    split_ratio: use to define spliting ratio for training and testing data default is 0.2 (20% of data is use for testing and 80% for training)
    """
    # Target value preprocessing
    df = df[[X_column_name,y_column_name]] # C3 for input column ,C5 target column
    df[y_column_name] = pd.Series(convert_to_number(df[y_column_name].to_list()),name=y_column_name)
    df = df[ df[y_column_name] <= 1]

    #input Value preprocessing
    #Step 1 remove html Tags an extra special character
    df[X_column_name] = remove_html_tags_special_character(df[X_column_name])
    df[X_column_name].replace(to_replace='\n',value='',inplace=True,regex=True)
    df[X_column_name].replace(to_replace='\\?',value='',inplace=True,regex=True)
    df[X_column_name].dropna(inplace=True)

    # Balance Data
    df = balance_data(df,y_column_name)
    # Step 3 NLP Text Preprocessing
    df[X_column_name] = df[X_column_name].apply(lambda x: remove_punctuations(x))
    df[X_column_name] = df[X_column_name].apply(lambda text: remove_stopwords(text))
    
    if len(split_ratio) == 2:
        train_x, test_x, train_y, test_y = train_test_split(df[X_column_name],df[y_column_name],test_size=split_ratio[1])
        train_x, val_x, train_y, val_y = train_test_split(train_x,train_y,test_size=split_ratio[0])
        return ((train_x,train_y), (val_x, val_y), (test_x, test_y))
    else:
        train_x, test_x, train_y, test_y = train_test_split(df[X_column_name],df[y_column_name],test_size=split_ratio[0])
        return ((train_x, train_y), (test_x, test_y))

## Data Loading and preprocessing

 1. Set the training dataset path
 2. Set the input colum name and target column name (all column name updated as C1, C2, C3 so if  input column is 3 so set  INPUT_COLUMN_NAME = C3 same for TARGET_COLUMN_NAME)
 3. set the training testing and validation ratio. i.e ([0.1,0.2], 10% data will use for validation, 20% data will use for testing and 70% data will use for training)

In [7]:
DATASET_PATH = r"A:\CJ_Personal\Upwork\Text Moderation\Dataset\messages.csv"
INPUT_COLUMN_NAME = "C3"
TARGET_COLUMN_NAME = "C5"
Max_COLUMN = 100
df = pd.read_csv(DATASET_PATH,encoding="UTF-8",names=[f"C{i}" for i in range(Max_COLUMN)])
VALIDATION_RATIO = 0.1
TESTING_RATIO = 0.2
train_set, val_set, test_set = preprocess(df,
                                          INPUT_COLUMN_NAME,
                                          TARGET_COLUMN_NAME,
                                          [VALIDATION_RATIO,TESTING_RATIO])

C:\Users\codej\AppData\Local\Temp\ipykernel_6444\3981657154.py:5: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATASET_PATH,encoding="UTF-8",names=[f"C{i}" for i in range(Max_COLUMN)])
C:\Users\codej\AppData\Local\Temp\ipykernel_6444\2625709502.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[y_column_name] = pd.Series(convert_to_number(df[y_column_name].to_list()),name=y_column_name)


(578489, 2)
(91143, 2)


KeyboardInterrupt: 

## Bert Model

### Bert Model Download

In [52]:
# Downloading Pre-build Bert Model Layer
## Variants of bert (https://tfhub.dev/google/collections/bert/1)

import tensorflow_hub as hub
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

### Bert Model Wrapper ( input and output layer added)

In [53]:
# bert model wth input and output layer wrapper

text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name = 'Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'Dense')(dropout)
# creating final model
model = tf.keras.Model(inputs = [text_input], outputs = [outputs])


### GPU configuration for model training ( run only system have good GPU Card)

In [54]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')
logical_gpus = tf.config.list_logical_devices('GPU')

In [55]:
## Model Summary to print
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 keras_layer_6 (KerasLayer)     {'input_type_ids':   0           ['Inputs[0][0]']                 
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

### Bert Model Compile with checkpoint callback and respective result metrics

In [56]:
# model Compiling

Metrics = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.Recall(name = 'recall')
           ]
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="/profane_Model CheckPoints",
                                                 save_weights_only=True,
                                                 verbose=1)
# compiling our model
model.compile(optimizer ='adam',
               loss = 'binary_crossentropy',
               metrics = Metrics)

### Model Training

In [58]:
history = model.fit(train_set[0], train_set[1],batch_size=32,validation_data=val_set,shuffle=True, epochs = 10)

Epoch 1/10
4148/4148 [==============================] - 2422s 584ms/step - loss: 0.2746 - accuracy: 0.8888 - precision: 0.8297 - recall: 0.5544 - val_loss: 0.2313 - val_accuracy: 0.9109 - val_precision: 0.8119 - val_recall: 0.7249
Epoch 2/10
4148/4148 [==============================] - 2418s 583ms/step - loss: 0.2385 - accuracy: 0.9054 - precision: 0.8197 - recall: 0.6714 - val_loss: 0.2291 - val_accuracy: 0.9078 - val_precision: 0.8906 - val_recall: 0.6173
Epoch 3/10
4148/4148 [==============================] - 2408s 581ms/step - loss: 0.2335 - accuracy: 0.9081 - precision: 0.8232 - recall: 0.6842 - val_loss: 0.2249 - val_accuracy: 0.9094 - val_precision: 0.8784 - val_recall: 0.6379
Epoch 4/10
4148/4148 [==============================] - 2406s 580ms/step - loss: 0.2331 - accuracy: 0.9079 - precision: 0.8197 - recall: 0.6876 - val_loss: 0.2163 - val_accuracy: 0.9138 - val_precision: 0.8702 - val_recall: 0.6713
Epoch 5/10
4148/4148 [==============================] - 2407s 580ms/step - l

### full Model Save

In [75]:
opt = tf.saved_model.SaveOptions(
    namespace_whitelist=None,
    save_debug_info=False,
    function_aliases=None,
    experimental_io_device="CPU:0",
    experimental_variable_policy=None,
    experimental_custom_gradients=True
)
model.save("models/profane_model",options=opt)

INFO:tensorflow:Assets written to: models/profane_model\assets


INFO:tensorflow:Assets written to: models/profane_model\assets


### Only Save model weight

In [77]:
model.save_weights("model_weight/profane_model_weight")

### Model Evalution

In [61]:
model.evaluate(test_set[0],test_set[1],batch_size=32)

1153/1153 [==============================] - 600s 521ms/step - loss: 0.2179 - accuracy: 0.9124 - precision: 0.9035 - recall: 0.6377


[0.21792438626289368,
 0.9124243855476379,
 0.9035120606422424,
 0.6377449035644531]

### Model Evalution  Report

In [68]:
from sklearn.metrics import precision_recall_fscore_support

pred = model.predict(test_set[0])
pred_int = tf.cast(pred, tf.int32)



1153/1153 [==============================] - 314s 273ms/step


In [69]:
precision_recall_fscore_support(test_set[1],pred_int)

C:\Users\codej\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.79650674, 0.        ]),
 array([1., 0.]),
 array([0.88672836, 0.        ]),
 array([29368,  7503], dtype=int64))